In [37]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, List
from shapely.geometry import Point, LineString, Polygon
from ipywidgets import interact, FloatSlider, Label
import scienceplots

# Exercise 2: **Synthetic Aperture Radar**

*Explain and visualize the frequency shift of a target as seen by a Synthetic Aperture Radar as it moves across its footprint.*

## *How does a SAR-system work?*

SAR offer a high spatial resolution through employin chirps, but at the cost of temporal resolution (low temporal revisit). As the satellite propagates along its track, contoniusly collecting information about the ground, a potential target will be captured in mutliple received signals. This is due to consecutive footprints overlapping. 
This basically translates into an antenna array along the satellite track. Digitally combining these antennas of real aperture results in a synthetic antenna of much larger (synthetic) aperture and thus resolution. Employing Doppler-shift processing, sub-pixel features can be identified by analyzing the combined received "synthetic" signal.

## *Which parameters inﬂuence the induced frequency-shift?*

If the moving SAR receives a signal from a target located ahead in the foorprint, relative to the SAR's current position, this signal is blue-shifted. Vide versa, a sginal stemming from a target from the opposite direction experiences a red-shift.

The change in frequency can be written as:

$f_v = f_0 - \dfrac{v_{rel}}{\lambda} = f_0 - \dfrac{v^2t}{R\lambda}$ 

Here, $f_0$ is the frequency of the transmitted signal, $\lambda$ its wavelength, $v$ the propagation velocity of the satellite and $R$ the slant range.
Thus, mainly the propagation velocity of the satellite contributes to the Doppler shift.

## *What are the beneﬁts of a SAR-system compared to a real-aperture radar?*

First and foremost, sufficiently high resolution with a real-aperture antenna can only be achieved with large antenna/aperture sizes. SAR bets on smaller hardware, thus saving on weight and cost.

## *How does the ground resolution of a SAR-system relate to the ﬂying height and the antenna size, as opposed to real-aperture antennas?*

For a real-aperture antenna, the azimuthal resolution was given as a function of the satellite's altitude, polar angle and antenna size:

$r_{a, real}\left(R, l\right)\approx R\dfrac{\lambda}{l} \approx \dfrac{H\lambda}{l}\tan\left(\theta\right)$

For a SAR, the azimuthal resolution is:

$r_{a, SAR}\left(l\right) = \dfrac{l}{2}$

Comparing the resolutions reveals, that while a real-aperture antenna either needs to fly very low or have a large antenna to minimize its azimuthal resolution. For SAR on the other hand, the azimuthal resolution is independent of altitude and actually gets better, the smaller the antenna.

## *(optional bonus-question) What do we mean by focused/unfocused SAR?*



In [44]:
SPEED_OF_LIGHT = 299792458 # m/s
def rel_freq(freq_0: float, velocity: float, distance: float, time: float) -> float:
    return freq_0 - ((velocity * velocity  * time) / (distance * (SPEED_OF_LIGHT / freq_0)))

In [89]:

freq_0_init = 7       # GHz
velocity_init = 6       # km/s
distance_init = 800     # km

# velocity_slider = FloatSlider(min=6, max=10, step=0.1, value=velocity_init, description='v [km/s]')
freq_0_slider = FloatSlider(min=0.1, max=10, step=0.1, value=freq_0_init, description='nu_0 [GHz]')
distance_slider = FloatSlider(min=200, max=1000, step=100, value=distance_init, description='R [km]')
time_slider = FloatSlider(min=0, max=1e6, step=1, value=0, description='t [s]')

@interact(time=time_slider, freq_0=freq_0_slider, distance=distance_slider)
def update(time, freq_0, distance):
    velocity = np.linspace(-10, 10, 1000)
    velocity *= 1e3
    # print(velocity)
    freq_0 *= 1e9
    distance *= 1e3

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111)
    ax.set_title(r'Relative frequency shift: $\nu_v = \nu_0 - \dfrac{v^2 t}{R\lambda}$' + '\n', fontsize = 16)
    ax.set_xlabel('Velocity v [km/s]', fontsize = 12)
    ax.set_ylabel(r'Relative frequency $\nu_v$ [GHz]', fontsize = 12)

    ax.plot(velocity, rel_freq(freq_0, velocity, distance, time) / 1e9, linewidth = 2, color = 'black',label = r'$\nu_v$ with $\nu_0 = $' + f'{(freq_0 / 1e9):.2f}' + ' GHz, ' + r'$R = $' + str(distance / 1e3) + ' km, ' + r'$t = $' + f'{time:.2e}' + ' s')
    ax.plot(velocity, [freq_0 / 1e9 for i in velocity], linewidth = 2, color = 'grey', linestyle = 'dashed',label = r'$\nu_0$')
    ax.tick_params(axis='both', direction='in', left = True, bottom = True, top = True, right=True)
    ax.set_ylim(0, 10.2)

    plt.legend(fontsize = 12)
    plt.tight_layout()


interactive(children=(FloatSlider(value=0.0, description='t [s]', max=1000000.0, step=1.0), FloatSlider(value=…

In [90]:
limit = 100
center_list = [i for i in range(limit)]
radii = [0 for i in center_list]
# print(radii)
time_slider = FloatSlider(value=-1, min=-1, max=limit, step=1, description=r't [a.u.]')
wave_propagation_velocity = FloatSlider(value=1, min=0, max=2, step=0.1, description=r'v_w/v_s')

@interact(t = time_slider, v = wave_propagation_velocity)
def update(t, v):
    fig = plt.figure(figsize = (16, 9))
    ax = fig.add_subplot(111)

    for idx in range(int(t) + 1):
        radii[idx] += v

    inner_radius = 0.01
    for idx, (outer_radius, center )in enumerate(zip(radii, center_list)):
        center_point = Point(center, 0)

        outer_circle = center_point.buffer(outer_radius)
        inner_circle = center_point.buffer(inner_radius)

        ring = outer_circle.difference(inner_circle)
        x, y = ring.exterior.xy
        if idx == 0:
            ax.plot(x, y, color = 'grey', label = 'Signal')
        else:
            ax.plot(x, y, color = 'grey')

        
    ax.plot(t, 0, linestyle = 'None', marker = 'o', color = 'black', label = 'Satellite')
    ax.set_aspect('equal')
    ax.set_title(f'Non-Scientific Sketch of Doppler Effect for a moving source:\nsignal propagation velocity = {v} * satellite velocity', fontsize = 16)
    ax.set_xlabel('Position [a.u.]', fontsize = 12)
    ax.set_ylabel('Position [a.u.]', fontsize = 12)
    ax.legend(loc = 'upper right')
    ax.set_xlim(-10, 60)
    ax.set_ylim(-5, 5)

    ax.set_xticklabels([])
    ax.set_yticklabels([])

    ax.tick_params(axis='both', direction='in', left = True, bottom = True, top = True, right=True)
    


interactive(children=(FloatSlider(value=-1.0, description='t [a.u.]', min=-1.0, step=1.0), FloatSlider(value=1…